# Sentiment Analyzer

* Data: Amazon reviews
* But the problem has been simplified, instead of 5 stars, the output is classified only as positive or negative
* The only input is the "review_text", all other data is ignored
* BeautifulSoup is a XML Parser

In [50]:
import nltk
import numpy as np
from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from bs4 import BeautifulSoup

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm

In [29]:
wordnet_lemmatizer = WordNetLemmatizer()

def my_tokenizer(sentence):
    sentence = sentence.lower() # All sentences are now in lower case
    tokens = nltk.tokenize.word_tokenize(sentence) # Creates tokens
    tokens = [t for t in tokens if len(t) > 2] # Ignore all tokens with only one letter
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] # Applies the lemmatizer
    tokens = [t for t in tokens if t not in stopwords.words('english') and t != ''] # Removes stop words and empty tokens
    return tokens

In [30]:
positive_reviews = BeautifulSoup(open('sorted_data_acl//electronics//positive.review').read())
positive_reviews = positive_reviews.findAll('review_text') # Uses only the review_text info

negative_reviews = BeautifulSoup(open('sorted_data_acl//electronics//negative.review').read())
negative_reviews = negative_reviews.findAll('review_text') # Uses only the review_text info

c:\users\sgtab\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file c:\users\sgtab\appdata\local\programs\python\python36\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [31]:
# Since we have more positive reviews than negatives, we trim the amount of positive reviews
np.random.shuffle(positive_reviews)
positive_reviews = positive_reviews[:len(negative_reviews)]

In [32]:
# Dictionary containing every word in the vocabulary
word_index_map = {}
current_index = 0

positive_tokenized = []
negative_tokenized = []

In [33]:
# Creates the dictionary: defines the vocabulary size and the index of each word
# The vocabulary could still be reduced

for review in positive_reviews:
    tokens = my_tokenizer(review.text)
    positive_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1
            
for review in negative_reviews:
    tokens = my_tokenizer(review.text)
    negative_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

print(word_index_map)     

{'great': 0, 'cable': 1, 'work': 2, 'advertised': 3, 'spend': 4, 'name': 5, 'brand': 6, 'performs': 7, 'better': 8, 'worse': 9, 'connected': 10, 'third': 11, 'gen': 12, 'mot': 13, '6412': 14, 'dvr': 15, 'sony': 16, '36xbr800': 17, 'picture': 18, 'transmit': 19, 'digital': 20, 'signal': 21, 'unlimited': 22, 'purchased': 23, 'iterm': 24, 'hp8450': 25, 'printer': 26, 'outstanding': 27, 'cartridge': 28, 'doe': 29, 'job.only': 30, 'problem': 31, 'get': 32, 'eaten': 33, 'time': 34, 'cost': 35, 'per': 36, 'print': 37, 'bargain': 38, "n't": 39, 'catridge': 40, 'process': 41, 'know': 42, 'ha': 43, 'reasonable': 44, 'price': 45, 'eat': 46, 'spit': 47, 'via': 48, 'cartidges': 49, 'real': 50, 'profit': 51, 'lie': 52, 'amazon': 53, 'top': 54, 'service': 55, 'delivery': 56, 'thing': 57, 'care': 58, 'wa': 59, 'design': 60, 'disk': 61, 'quality': 62, 'dvd': 63, 'bought': 64, 'based': 65, 'overall': 66, 'positive': 67, 'review': 68, 'good': 69, 'sound': 70, 'comfortable': 71, 'fit': 72, 'find': 73, 'ti

In [34]:
def tokens_to_vector(tokens, label):
    # Feature vector has the same size of the vocabulary (+1 for the label) and has all values set to 0
    x = np.zeros(len(word_index_map) + 1) 
    # Find the word in the dictionary, retrieve its index 
    # Change the value of the word for this particular X vector
    for t in tokens:
        i = word_index_map[t]
        x[i] += 1
    # Normalizes X in relation to the total amount of words in the document
    x = x / x.sum()
    x[-1] = label
    return x

In [35]:
N = len(positive_tokenized) + len(negative_tokenized)
data = np.zeros((N, len(word_index_map) + 1))
i = 0
                
for tokens in positive_tokenized:
    xy = tokens_to_vector(tokens, 1)
    data[i,:] = xy
    i += 1
    
for tokens in negative_tokenized:
    xy = tokens_to_vector(tokens, 0)
    data[i,:] = xy
    i += 1
    
print("Rows (input size): ", len(data))
print("Columns (feature vector size): ", len(data[0]))

Rows (input size):  2000
Columns (feature vector size):  11289


In [36]:
np.random.shuffle(data)

In [43]:
# Split data
# You can manually split or use sklearn function

#X = data[:, :-1]
#Y = data[:, -1:]

#X_train = X[:-100,]
#Y_train = Y[:-100,]

#X_test = X[-100:,]
#Y_test = Y[-100:,]


X_train, X_test, Y_train, Y_test = train_test_split(data[:, :-1], 
                                                    data[:, -1:], 
                                                    test_size = 0.25, 
                                                    random_state = 42, 
                                                    shuffle = True)

In [44]:
# Linear Regression
model = LogisticRegression()
model.fit(X_train, Y_train)
print("Classification rate: ", model.score(X_test, Y_test))

Classification rate:  0.728


c:\users\sgtab\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [45]:
# MLP with two layers
mlp_model = MLPClassifier(hidden_layer_sizes = (30, 15, ), 
                          activation = "relu", 
                          solver = "adam", 
                          alpha = 0.65, 
                          max_iter = 1000, 
                          random_state = 1234)

mlp_model.fit(X_train, Y_train)
print("Classification rate: ", mlp_model.score(X_test, Y_test))

c:\users\sgtab\appdata\local\programs\python\python36\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Classification rate:  0.822
